## 概要
 1. language modelsを使ってみる
 2. OutputParsersを使ってみる
 3. PromptTemplateを使ってみる
 4. LCELを使ってみる
 5. 会話履歴を使ってみる

### リンク
 - <a href="https://python.langchain.com/v0.2/docs/tutorials/llm_chain/" target=_blank>Langchain Tutorial</a>
 - <a href="https://pypi.org/project/langchain/#history" target=_black>Langchain Version</a>


In [ ]:
import os
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)
from langchain_core.messages import (
    HumanMessage, 
    AIMessage,
    SystemMessage
)
from dotenv import load_dotenv
load_dotenv('../.env')


### 1. language modelsを使ってみる

#### 1-1. langchainを使わない場合

 - <a href="https://learn.microsoft.com/ja-jp/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python" target=_blank>モデルの実行</a>
 - <a href="https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release" target=_blank>Azure API Versionの確認</a>
 - <a href="https://platform.openai.com/docs/models" target=_blank>OpenAIのモデルリスト</a>

In [ ]:
import os
from openai import (
    OpenAI,
    AzureOpenAI
)
# Azureの場合
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    client = AzureOpenAI(api_version="2024-06-01")
    model_name = "chat" # Azureでデプロイしたモデル名
# OpenAIの場合
elif os.getenv('OPENAI_API_KEY') != "":
    client = OpenAI()
    model_name = "gpt-4" # OpenAIのモデル名 
else:
    print("APIKeyの設定を確認してください")

In [ ]:
response = client.chat.completions.create(
    model=model_name, 
    # ロールはsystem, user, assistantを指定する。contentはユーザーが入力したテキスト。
    messages=[
        {"role": "system", "content": "あなたは役に立つアシスタントだ。"},
        {"role": "user", "content": "Azure OpenAIはcustomer managed keysをサポートしていますか？"},
        {"role": "assistant", "content": "はい、customer managed keysはAzure OpenAIでサポートされています。"},
        {"role": "user", "content": "他のAzure AIサービスもこれをサポートしていますか？"}
    ]
)

print(response.choices[0].message.content)

#### 1-2. langchainを使う場合

<a href="https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#using-language-models" target=_blank>モデルの実行</a>

In [ ]:
# LLMの定義
model = None
# Azureの場合
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    model = AzureChatOpenAI(
        azure_deployment="chat",
        openai_api_version="2024-06-01"
    )
# OpenAIの場合
elif os.getenv('OPENAI_API_KEY') != "":
    model = ChatOpenAI(model="gpt-4")
else:
    print("APIKeyの設定を確認してください")

In [ ]:
# langchainでは、SystemMessage, HumanMessage, AIMessageの3つのメッセージタイプを使用します。
messages = [
    SystemMessage(content="あなたは役に立つアシスタントだ。"),
    HumanMessage(content="Azure OpenAIはcustomer managed keysをサポートしていますか？"),
    AIMessage(content="はい、customer managed keysはAzure OpenAIでサポートされています。"),
    HumanMessage(content="他のAzure AIサービスもこれをサポートしていますか？")
]

result = model.invoke(messages)
print(result)

#### 補足：会話スタイルとパラメータ設定

| 会話スタイル | Temperature | Top P | 説明 |
|--------------|--------------|-------|------|
| **創造的に** | 0.7 - 1.0    | 0.9   | 高いtemperatureはモデルが多様な出力を生成しやすくし、創造的な応答を促します。Top Pも高めに設定して、広範な単語選択を可能にします。 |
| **バランスよく** | 0.5 - 0.7    | 0.8   | 中程度のtemperatureはバランスの取れた応答を生成し、創造性と一貫性のバランスを保ちます。Top Pも中程度に設定します。 |
| **厳密に** | 0.0 - 0.3    | 0.7   | 低いtemperatureはモデルが最も確率の高い単語を選びやすくし、厳密で一貫性のある応答を生成します。Top Pも低めに設定して、確定的な出力を促します。 |

``` Python

model = AzureChatOpenAI(
        azure_deployment="chat",
        openai_api_version="2024-06-01",
        temperature=0.5,
        top_p=1.0
    )

```


### 2. OutputParsersを使ってみる
AIの回答のフォーマットを指定して取得する

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser = StrOutputParser()
parser.invoke(result)

In [ ]:
# contentからも取得できる
print(result.content)

### 3. PromptTemplateを使ってみる

#### 3-1. PromptTemplateを使わない場合

In [ ]:
messages = [
    SystemMessage(content="イタリアの言語に翻訳してください"),
    HumanMessage(content="こんにちは"),
]
model.invoke(messages)

In [ ]:
messages = [
    SystemMessage(content="英語の言語に翻訳してください"),
    HumanMessage(content="こんばんわ"),
]
model.invoke(messages)

この2つの例を見ると、例えば
 - イタリア、英語などの言語を変更したい
 - 翻訳する対象（Humanmessage）を変更したい

ということもある。  

これを解決するのが、PromptTemplateの機能

#### 3-2. PromptTemplateを使う場合

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# プロンプトの定義
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{language}の言語に翻訳してください:"), 
        ("user", "{text}")
    ]
)

In [ ]:
# 辞書型で{}に入れる値を指定
prompt = prompt_template.invoke({"language": "italian", "text": "こんにちは"})
prompt

In [ ]:
# メッセージだけを抜粋
prompt.to_messages()

In [ ]:
# Promptを使ってみる
result = model.invoke(prompt)
print(result.content)

### 4. LCELを使ってみる
 今まで実行してきたように、Promtを設定 > モデルから回答取得 > 出力を成形と処理が続く  
 これをLCELと表記で、続けて記述することができます。  
 Chainという考え方になります。

#### 4-1. LCEL(Chain)を使わない場合

In [ ]:
# PromptTemplateでPromptを作成
prompt = prompt_template.invoke({"language": "italian", "text": "こんにちは"})
# modelにPromptを渡して結果を取得
response = model.invoke(prompt)
# 結果を文字列に変換
result = parser.invoke(response)
print(result)

#### 4-2. LCEL(Chain)を使わない場合

In [ ]:
# |を使って、PromptTemplate, ChatOpenAI, StrOutputParserをつなげる
chain = prompt_template | model | parser

In [ ]:
# あとは、chainを使って、結果を取得
chain.invoke({"language": "italian", "text": "こんにちは"})

### 5. 会話履歴を使ってみる
チャットでは会話履歴が大事になってきます。  
会話履歴の取り扱いについて習得していきましょう

#### 5-1. 会話履歴の必要性
なぜ、会話の履歴が必要か確認していきます

In [ ]:
# 1つ目のメッセージをいれてみます
model.invoke([HumanMessage(content="こんにちは、もものきです")])

In [ ]:
# 2つ目のメッセージをいれてみます
model.invoke([HumanMessage(content="私の名前を覚えていますか？")])

このように、1つずつメッセージを入れても、modelは適切な回答をしてくれません。  
過去の会話も続けて、modelに入れるとチャットのようなつづけた会話ができます

In [ ]:
model.invoke(
    [
        HumanMessage(content="こんにちは、もものきです"),
        AIMessage(content='こんにちは、もものきさん。ご用件はありますか？'),
        HumanMessage(content="私の名前を覚えていますか？"),
    ]
)

#### 5-2. 会話履歴 その1(<a href="https://python.langchain.com/v0.2/docs/tutorials/chatbot/#message-history" target=_blank>v0.2 公式ドキュメント</a>)

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

# 会話履歴置き場
store = {}

# セッションIDを指定して履歴を取得
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 履歴を取得するためのRunnableWithMessageHistoryを作成
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
# session_idのConfigを作成
# session_idは新しい会話を開始するたびに変更したりすると、会話ごとの履歴を保持することができます。
config = {"configurable": {"session_id": "abc123"}}

In [ ]:
# message_historyを使って1つ目の質問を投げてみる
response = with_message_history.invoke(
    [HumanMessage(content="こんにちは、もものきです")],
    config=config,
)
response.content

In [ ]:
# message_historyを使って2つ目の質問を投げてみる
response = with_message_history.invoke(
    [HumanMessage(content="私の名前を覚えていますか？")],
    config=config,
)

response.content

#### 5-3. 会話履歴その2 ConversationBufferMemoryなど
以前の方法

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
chain = ConversationChain(
     llm=model,
     memory=ConversationBufferMemory()
)

In [ ]:
chain.run([HumanMessage(content="こんにちは、もものきです")])

In [ ]:
chain.run([HumanMessage(content="私の名前を覚えていますか？")])

#### 5-4. 会話履歴その3 スクラッチ
langchainは便利な機能がある一方、後ろで何をしているかわかりにくい場合もあります。  
また、更新が多いライブラリであり、方法が変わっていきます。  
次のコードでメッセージ履歴のイメージを把握しましょう！

In [ ]:
# 会話履歴
messages = []

In [ ]:
messages.append(HumanMessage(content="こんにちは、もものきです"))
response = model.invoke(messages)
messages.append(AIMessage(content=response.content))
response

In [ ]:
messages.append(HumanMessage(content="私の名前を覚えていますか？"))
response = model.invoke(messages)
messages.append(AIMessage(content=response.content))
response

In [ ]:
messages

#### 5-5. 会話履歴管理
  
会話を続けると、履歴が長くなり、modelに入れるトークン数を超えたり、トークン数によって費用が決まるため、費用がかさむ場合があります。  
  
そのため、会話履歴は管理する必要があります。  
  
5-3. 会話履歴その2で紹介したMemoryには、ConversationSummaryMemory, ConversationBufferWindowMemoryなどで会話履歴のトークン数を調整する方法があります。  
  

直近のドキュメントでは、trim_messagesなどがありますが、シンプルに配列で調整してもよい

In [ ]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
]
messages


In [ ]:
new_message = [
    messages[0], # system message
    messages[-3], # human message
    messages[-2], # ai message
    messages[-1] # 最新 human message
]
new_message

In [ ]:
k = 3
new_message = [messages[0]]
new_message.extend(messages[-k:])
new_message

In [ ]:
response = model.invoke(new_message)
response